In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/02/19 21:21:36 WARN Utils: Your hostname, codespaces-876b42 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/02/19 21:21:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/19 21:21:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [35]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet

--2024-02-19 21:40:54--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2021-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.32.192.2, 13.32.192.124, 13.32.192.190, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.32.192.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21686067 (21M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2021-01.parquet’

yellow_tripdata_202 100%[===================>]  20.68M  58.5MB/s    in 0.4s    

2024-02-19 21:40:55 (58.5 MB/s) - ‘yellow_tripdata_2021-01.parquet’ saved [21686067/21686067]



In [36]:
!wc -l yellow_tripdata_2021-01.parquet

69649 yellow_tripdata_2021-01.parquet


In [38]:
import sys
import pandas as pd
!{sys.executable} -m pip install pyarrow
parquet_file = 'yellow_tripdata_2021-01.parquet'
df = pd.read_parquet(parquet_file)
csv_output = 'fhvhv_tripdata_2021-01.csv'
df.to_csv(csv_output, index=False, sep='\t')


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [39]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [40]:
df.schema

StructType([StructField('VendorID	tpep_pickup_datetime	tpep_dropoff_datetime	passenger_count	trip_distance	RatecodeID	store_and_fwd_flag	PULocationID	DOLocationID	payment_type	fare_amount	extra	mta_tax	tip_amount	tolls_amount	improvement_surcharge	total_amount	congestion_surcharge	airport_fee', StringType(), True)])

In [41]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [26]:
import pandas as pd

In [42]:
df_pandas = pd.read_csv('head.csv')

In [43]:
df_pandas.dtypes

VendorID\ttpep_pickup_datetime\ttpep_dropoff_datetime\tpassenger_count\ttrip_distance\tRatecodeID\tstore_and_fwd_flag\tPULocationID\tDOLocationID\tpayment_type\tfare_amount\textra\tmta_tax\ttip_amount\ttolls_amount\timprovement_surcharge\ttotal_amount\tcongestion_surcharge\tairport_fee    object
dtype: object

In [ ]:
spark.createDataFrame(df_pandas).schema

In [ ]:
from pyspark.sql import types

In [ ]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [ ]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [ ]:
df = df.repartition(24)

In [ ]:
df.write.parquet('fhvhv/2021/01/')

In [ ]:
df = spark.read.parquet('fhvhv/2021/01/')

In [ ]:
df.printSchema()

In [ ]:
from pyspark.sql import functions as F

In [ ]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [ ]:
crazy_stuff('B02884')

In [ ]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [ ]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

In [ ]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003')

In [ ]:
!head -n 10 head.csv